# Experiment Chooser
This widget will allow you to select an experiment directory and persist your choice across notebook boundaries. That means you only need to select your experiment once even if you're running analyses in multiple notebooks.

The widget accomplishes this behavior through usage of the %store magic command. When you initially select an experiment, the path to that experiment will be persisted globally by calling %store experiment_base. You can retrieve the selected path by calling %store -r experiment_base to import that variable to your local namespace. The widget will attempt to retrieve experiment_base whenever it is rendered, and if found it will render with experiment_base already selected.

call it with the magic command `%run experiment_chooser.ipynb`

In [ ]:
from ipyfilechooser import FileChooser
from common.utils import try_load_mfile

import pathlib

EXPERIMENT_DIRECTORY = "/nfs/turbo/umms-sbarmada/experiments"

def verify_experiment_dir(chooser):
    path = pathlib.Path(fc.selected_path)
    try:
        # magic command resolution is weird so we have to define these vars as globals and use them outside the try/except
        global spec; spec = try_load_mfile(path)
        global experiment_base; experiment_base = path
        chooser.title = ""
    except Exception as e:
        chooser.title = f'<b style="color:red">Error: {e}</b>'
    %store spec
    %store experiment_base

fc = FileChooser(EXPERIMENT_DIRECTORY, show_only_dirs=True)
fc.title = "<b>Select experiment directory</b>"
fc.register_callback(verify_experiment_dir)

try:
    %store -r experiment_base
    fc.default_path = experiment_base
    fc.title = ""
    fc._apply_selection()
except NameError:
    pass

display(fc)